In [104]:
print("RandomForest:")
print("rf map5:",map5eval(rf_preds[20000:30000],y_test[20000:30000]))
print("XGBoost:")
print("xgb map5:",map5eval(xgb_preds[20000:30000],y_test[20000:30000]))
print("adaboost:")
print("ada map5:",map5eval(ada_preds[20000:30000],y_test[20000:30000]))
# print("RandomForest:")
# print("rf map5:",map5eval(rf_preds,y_test))
# print("XGBoost:")
# print("xgb map5:",map5eval(xgb_preds,y_test))
# print("adaboost:")
# print("ada map5:",map5eval(ada_preds,y_test))

RandomForest:
recommendation 1 : 3607
recommendation 2 : 891
recommendation 3 : 614
recommendation 4 : 414
recommendation 5 : 324
rf map5: 0.4425466666666667
XGBoost:
recommendation 1 : 3081
recommendation 2 : 953
recommendation 3 : 635
recommendation 4 : 462
recommendation 5 : 361
xgb map5: 0.39568666666666663
adaboost:
recommendation 1 : 3767
recommendation 2 : 837
recommendation 3 : 549
recommendation 4 : 381
recommendation 5 : 291
ada map5: 0.45219499999999996


In [ ]:
import pickle
import numpy as np

def map5eval(preds, actual):
    predicted = []
    for i in range(preds.shape[0]):
        predicted.append(preds[i].argsort()[-5:][::-1])
    predicted = np.array(predicted)
    metric = 0.
    for i in range(5):
        metric += np.sum(actual==predicted[:,i])/(i+1)
        print("recommendation",i+1,":",np.sum(actual==predicted[:,i]))
    metric /= actual.shape[0]
    return metric

def map5eval2(predicted, actual):
    metric = 0.
    for i in range(5):
        metric += np.sum(actual==predicted[:,i])/(i+1)
#         print("recommendation",i+1,":",np.sum(actual==predicted[:,i]))
    metric /= actual.shape[0]
    return metric

xgb_preds = []
with open("XGB_PREDICTIONS_2","rb") as file:
    # xgb_preds = pickle.load(file)
    xgb_preds = pickle.load(file)
rf_preds = []
with open("RF_PREDICTIONS_FINALLL","rb") as file:
    rf_preds = pickle.load(file)
ada_preds = []
with open("ADA_PREDICTIONS_md_20_nest_500","rb") as file:
	ada_preds = pickle.load(file)
y_test = []
with open("y_test","rb") as file:
    y_test = np.array(pickle.load(file))

# w1 = 2.3 # rf weight - 4.2: 0.4542 --> 4.23
# w2 = 2 # xgb weight - 2: 0.455
# w3 = 2.5 # ada weight
# w1 = 2.5
# w2 = 1.5
# w3 = 4
import time
tm=0
wm=[]
weights = []
test_map5s = []
train_map5s = []
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
rf_preds2 = rf_preds
xgb_preds2 = xgb_preds
ada_preds2 = ada_preds
for train_index, test_index in sss.split(rf_preds2,y_test):
#     print("TRAIN:", train_index, "TEST:", test_index)
    w1 = 2
    w2 = 3
    w3 = 6
#     rf_preds, y_preds = rf_preds2[train_index], y_test[test_index]
#     xgb_preds, y_preds2 = xgb_preds2[train_index], y_test[test_index]
#     ada_preds, y_preds3 = ada_preds2[train_index], y_test[test_index]
    finprobs = []
    for i in train_index:
        p1 = rf_preds[i].argsort()[-5:][::-1]
        p2 = xgb_preds[i].argsort()[-5:][::-1]
        p3 = ada_preds[i].argsort()[-5:][::-1]
        p1_scores = sorted(rf_preds[i])[-5:][::-1]
        p2_scores = sorted(xgb_preds[i])[-5:][::-1]
        p3_scores = sorted(ada_preds[i])[-5:][::-1]
        probs = [0]*100
        for k in range(5):
            if p1[k]==p2[k] and p1[k] == p3[k]:
                probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k]) + (w3*p3_scores[k])
            elif p1[k]==p2[k]:
                probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k])
                probs[p3[k]] += (w3*p3_scores[k])
            elif p2[k]==p3[k]:
                probs[p2[k]] += (w2*p2_scores[k]) + (w3*p3_scores[k])
                probs[p1[k]] += (w1*p1_scores[k])
            elif p1[k]==p3[k]:
                probs[p1[k]] += (w1*p1_scores[k]) + (w3*p3_scores[k])
                probs[p2[k]] += (w2*p2_scores[k])
            else:
                probs[p1[k]] += (w1*p1_scores[k])
                probs[p2[k]] += (w2*p2_scores[k])
                probs[p3[k]] += (w3*p3_scores[k])

        probs = np.array(probs).argsort()[-5:][::-1]
        finprobs.append(probs)
    trmap5 = map5eval2(np.array(finprobs),y_test[train_index])
    print("train ensemble map5:",trmap5)
    print(np.array(finprobs).shape,y_test[train_index].shape)
    finprobs = []
    for i in test_index:
        p1 = rf_preds[i].argsort()[-5:][::-1]
        p2 = xgb_preds[i].argsort()[-5:][::-1]
        p3 = ada_preds[i].argsort()[-5:][::-1]
        p1_scores = sorted(rf_preds[i])[-5:][::-1]
        p2_scores = sorted(xgb_preds[i])[-5:][::-1]
        p3_scores = sorted(ada_preds[i])[-5:][::-1]
        probs = [0]*100
        for k in range(5):
            if p1[k]==p2[k] and p1[k] == p3[k]:
                probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k]) + (w3*p3_scores[k])
            elif p1[k]==p2[k]:
                probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k])
                probs[p3[k]] += (w3*p3_scores[k])
            elif p2[k]==p3[k]:
                probs[p2[k]] += (w2*p2_scores[k]) + (w3*p3_scores[k])
                probs[p1[k]] += (w1*p1_scores[k])
            elif p1[k]==p3[k]:
                probs[p1[k]] += (w1*p1_scores[k]) + (w3*p3_scores[k])
                probs[p2[k]] += (w2*p2_scores[k])
            else:
                probs[p1[k]] += (w1*p1_scores[k])
                probs[p2[k]] += (w2*p2_scores[k])
                probs[p3[k]] += (w3*p3_scores[k])

        probs = np.array(probs).argsort()[-5:][::-1]
        finprobs.append(probs)
    tmap5 = map5eval2(np.array(finprobs),y_test[train_index])
    if tmap5 > tm:
        tm = tmap5
        wm = [w1,w2,w3]
    weights.append([w1,w2,w3])
    test_map5s.append(tmap5)
    train_map5s.append(trmap5)
    print(np.array(finprobs).shape,y_test[test_index].shape)
    print("test ensemble map5:",map5eval2(np.array(finprobs),y_test[test_index]))
    print("RandomForest:")
    print("rf map5:",map5eval(rf_preds[test_index],y_test[test_index]))
    print("XGBoost:")
    print("xgb map5:",map5eval(xgb_preds[test_index],y_test[test_index]))
    print("adaboost:")
    print("ada map5:",map5eval(ada_preds[test_index],y_test[test_index]))
    print("------")
print(wm,tm)

## Voting scheme - RandomForest, XGBoost, AdaBoost

In [ ]:
RandomForest:
recommendation 1 : 10985
recommendation 2 : 2695
recommendation 3 : 1843
recommendation 4 : 1308
recommendation 5 : 967
rf map5: 0.4489077777777778
XGBoost:
recommendation 1 : 9420
recommendation 2 : 2857
recommendation 3 : 1908
recommendation 4 : 1366
recommendation 5 : 1066
xgb map5: 0.4013066666666667
adaboost:
recommendation 1 : 11529
recommendation 2 : 2497
recommendation 3 : 1613
recommendation 4 : 1104
recommendation 5 : 884
ada map5: 0.4589322222222222
Ensemble:
recommendation 1 : 11445
recommendation 2 : 2739
recommendation 3 : 1676
recommendation 4 : 1316
recommendation 5 : 915
ensemble map5: 0.46283888888888886

In [1]:
import pickle
import numpy as np

def map5eval(preds, actual):
    predicted = []
    for i in range(preds.shape[0]):
        predicted.append(preds[i].argsort()[-5:][::-1])
    predicted = np.array(predicted)
    metric = 0.
    for i in range(5):
        metric += np.sum(actual==predicted[:,i])/(i+1)
        print("recommendation",i+1,":",np.sum(actual==predicted[:,i]))
    metric /= actual.shape[0]
    return metric

def map5eval2(predicted, actual):
    metric = 0.
    for i in range(5):
        metric += np.sum(actual==predicted[:,i])/(i+1)
        print("recommendation",i+1,":",np.sum(actual==predicted[:,i]))
    metric /= actual.shape[0]
    return metric

xgb_preds = []
with open("XGB_PREDICTIONS_2","rb") as file:
    # xgb_preds = pickle.load(file)
    xgb_preds = pickle.load(file)
rf_preds = []
with open("RF_PREDICTIONS_FINALLL","rb") as file:
    rf_preds = pickle.load(file)
ada_preds = []
with open("ADA_PREDICTIONS_md_20_nest_500","rb") as file:
	ada_preds = pickle.load(file)
y_test = []
with open("y_test","rb") as file:
    y_test = np.array(pickle.load(file))

# print("RandomForest:")
# print("rf map5:",map5eval(rf_preds,y_test))
# print("XGBoost:")
# print("xgb map5:",map5eval(xgb_preds,y_test))
# print("adaboost:")
# print("ada map5:",map5eval(ada_preds,y_test))
# w1 = 2.3 # rf weight - 4.2: 0.4542 --> 4.23
# w2 = 2 # xgb weight - 2: 0.455
# w3 = 2.5 # ada weight
# w = [[3,4,6],[3,4,7],[3,4,8],[3,4,9],[3,4,10],[3,4,11],[3,4,12],[3,4,13]]
# print("xgb map5:",map5eval(xgb_preds[20000:30000],y_test[20000:30000]))
w1 = 2
w2 = 3
w3 = 6
print("w1:",w1,"w2:",w2,"w3:",w3)
finprobs=[]
test_ops = []
test_preds = []
for i in range(20000,30000):
    p1 = rf_preds[i].argsort()[-5:][::-1]
    p2 = xgb_preds[i].argsort()[-5:][::-1]
    p3 = ada_preds[i].argsort()[-5:][::-1]
    p1_scores = sorted(rf_preds[i])[-5:][::-1]
    p2_scores = sorted(xgb_preds[i])[-5:][::-1]
    p3_scores = sorted(ada_preds[i])[-5:][::-1]
    l = []
    l.extend(p1)
    l.extend(p2)
    l.extend(p3)
    l.extend(p1_scores)
    l.extend(p2_scores)
    l.extend(p3_scores)
    
    test_preds.append(l)
    test_ops.append(int(y_test[i]))
    probs = [0]*100
    for k in range(5):
        if p1[k]==p2[k] and p1[k] == p3[k]:
            probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k]) + (w3*p3_scores[k])
        elif p1[k]==p2[k]:
            probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k])
            probs[p3[k]] += (w3*p3_scores[k])
        elif p2[k]==p3[k]:
            probs[p2[k]] += (w2*p2_scores[k]) + (w3*p3_scores[k])
            probs[p1[k]] += (w1*p1_scores[k])
        elif p1[k]==p3[k]:
            probs[p1[k]] += (w1*p1_scores[k]) + (w3*p3_scores[k])
            probs[p2[k]] += (w2*p2_scores[k])
        else:
            probs[p1[k]] += (w1*p1_scores[k])
            probs[p2[k]] += (w2*p2_scores[k])
            probs[p3[k]] += (w3*p3_scores[k])

    probs = np.array(probs).argsort()[-5:][::-1]
    finprobs.append(probs)
trmap5 = map5eval2(np.array(finprobs),y_test[20000:30000])
print("map5:",trmap5)
finprobs = []
train_preds = []
train_ops = []
for i in range(20000):
    p1 = rf_preds[i].argsort()[-5:][::-1]
    p2 = xgb_preds[i].argsort()[-5:][::-1]
    p3 = ada_preds[i].argsort()[-5:][::-1]
    p1_scores = sorted(rf_preds[i])[-5:][::-1]
    p2_scores = sorted(xgb_preds[i])[-5:][::-1]
    p3_scores = sorted(ada_preds[i])[-5:][::-1]
    l = []
    l.extend(p1)
    l.extend(p2)
    l.extend(p3)
    l.extend(p1_scores)
    l.extend(p2_scores)
    l.extend(p3_scores)
    
    train_preds.append(l)
    train_ops.append(int(y_test[i]))
    probs = [0]*100
    for k in range(5):
        if p1[k]==p2[k] and p1[k] == p3[k]:
            probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k]) + (w3*p3_scores[k])
        elif p1[k]==p2[k]:
            probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k])
            probs[p3[k]] += (w3*p3_scores[k])
        elif p2[k]==p3[k]:
            probs[p2[k]] += (w2*p2_scores[k]) + (w3*p3_scores[k])
            probs[p1[k]] += (w1*p1_scores[k])
        elif p1[k]==p3[k]:
            probs[p1[k]] += (w1*p1_scores[k]) + (w3*p3_scores[k])
            probs[p2[k]] += (w2*p2_scores[k])
        else:
            probs[p1[k]] += (w1*p1_scores[k])
            probs[p2[k]] += (w2*p2_scores[k])
            probs[p3[k]] += (w3*p3_scores[k])

    probs = np.array(probs).argsort()[-5:][::-1]
    finprobs.append(probs)
trmap5 = map5eval2(np.array(finprobs),y_test[:20000])
print("map5:",trmap5)

recommendation 1 : 3081
recommendation 2 : 953
recommendation 3 : 635
recommendation 4 : 462
recommendation 5 : 361
xgb map5: 0.39568666666666663
w1: 2 w2: 3 w3: 6
recommendation 1 : 3802
recommendation 2 : 882
recommendation 3 : 542
recommendation 4 : 398
recommendation 5 : 320
map5: 0.4587166666666667
recommendation 1 : 7776
recommendation 2 : 1785
recommendation 3 : 1095
recommendation 4 : 808
recommendation 5 : 601
map5: 0.46778500000000006


In [11]:
# w1: 1 w2: 1 w3: 1
# map5: 0.46283888888888886
# to beat: 0.4589

In [37]:
print(len(train_preds[0]),len(train_ops))

18 20000


In [28]:
print(len(test_preds[0]),len(test_ops))
def map5eval(preds, actual):
    predicted = []
    for i in range(preds.shape[0]):
        predicted.append(preds[i].argsort()[-5:][::-1])
    predicted = np.array(predicted)
    metric = 0.
    for i in range(5):
        metric += np.sum(actual==predicted[:,i])/(i+1)
#         print("recommendation",i+1,":",np.sum(actual==predicted[:,i]))
    metric /= actual.shape[0]
    return metric

30 10000


In [33]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=36,n_estimators=300, n_jobs=-1, warm_start=True)
op = np.array(train_ops)
tr = np.array(train_preds)
print("fitting RF model...")
clf.fit(tr[:15000],op[:15000])

fitting RF model...


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=36, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=True)

In [34]:
map5eval(clf.predict_proba(np.array(train_preds[15000:20000])),np.array(train_ops[15000:20000]))

0.40545

In [35]:
map5eval(rf_preds[15000:20000],np.array(train_ops[15000:20000]))

0.4506

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
pipe = Pipeline([('classifier' , RandomForestClassifier())])
param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['lbfgs']},
    {'classifier' : [RandomForestClassifier(max_depth=10,n_estimators=100, n_jobs=-1, warm_start=True)],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
]
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)
best_clf = clf.fit(tr[:12000],op[:12000])


fitting..
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 13.3min finished


In [13]:
map5eval(best_clf.predict_proba(np.array(train_preds[12000:])),np.array(train_ops[12000:]))

recommendation 1 : 2866
recommendation 2 : 387
recommendation 3 : 285
recommendation 4 : 243
recommendation 5 : 183


0.40648125

In [23]:
map5eval(best_clf.predict_proba(np.array(test_preds)),np.array(test_ops))

0.3982716666666667

In [32]:
map5eval(rf_preds[20000:30000],np.array(test_ops))

0.4425466666666667